In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-10-25 18:08:59.518896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761415739.709935      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761415739.764706      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Abstractive summarization with Flan T5 Small

## Load and inspect dataset

In [4]:
datasets = load_dataset("knkarthick/xsum")

README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/480M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/26.3M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/26.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 11334
    })
})

In [7]:
datasets["train"] = datasets["train"].select(range(10000))
datasets["validation"] = datasets["validation"].select(range(1000))
datasets["test"] = datasets["test"].select(range(1000))

In [8]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 1000
    })
})

## Preprocess data

In [9]:
datasets = datasets.filter(lambda x: x["dialogue"] is not None and len(x["dialogue"].strip()) > 0)

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
checkpoint = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [11]:
max_input_length = 512
max_target_length = 150

def preprocess(example):
    inputs = ["summarize: " + doc for doc in example["dialogue"]]  # with flan-t5 we need to instruct it with the task
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(text_target=example["summary"], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = datasets.map(preprocess, batched=True, remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/9998 [00:00<?, ? examples/s]

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 9998
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 999
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

## Load model

In [13]:
checkpoint = "google/flan-t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Fine tune model

In [14]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
from nltk.tokenize import sent_tokenize

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
rouge_score = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value * 100 for key, value in result.items()}  # Convert to percentage
    return {k: round(v, 4) for k, v in result.items()}

In [17]:
import os
os.environ["WANDB_DISABLED"] = "true"

batch_size = 8
logging_steps = len(tokenized_datasets["train"]) // batch_size

# training_args = Seq2SeqTrainingArguments(
#     output_dir="./flan-t5-xsum",
#     eval_strategy="epoch",
#     learning_rate=2e-4,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     weight_decay=0.01,
#     save_total_limit=2,
#     num_train_epochs=3,
#     predict_with_generate=True,
#     # fp16=True,
#     logging_steps=logging_steps,
#     push_to_hub=False,
#     report_to="none"
# )

training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-xsum",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=False,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# trainer.train(resume_from_checkpoint = True)
trainer.train()

/tmp/ipykernel_37/183533910.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.606800,2.309609,29.722700,8.744100,23.735800,23.698800
2,2.539700,2.301565,29.976500,8.954200,23.859600,23.833700
3,2.491800,2.297836,30.097500,9.057700,24.002700,23.972400
4,2.454000,2.297060,30.150400,8.879900,23.971100,23.938900
5,2.433000,2.295445,29.860400,8.772000,23.878800,23.856400


TrainOutput(global_step=6250, training_loss=2.5051662254333498, metrics={'train_runtime': 1543.2517, 'train_samples_per_second': 32.393, 'train_steps_per_second': 4.05, 'total_flos': 9271959870652416.0, 'train_loss': 2.5051662254333498, 'epoch': 5.0})

In [25]:
from transformers import pipeline
summarizer = pipeline("text2text-generation",
                      model="/kaggle/working/flan-t5-xsum/checkpoint-6250",
                      max_length=100,
                      min_length=50,)
summarizer_base = pipeline("text2text-generation", model="google/flan-t5-small")

Device set to use cuda:0
Device set to use cuda:0


In [26]:
def print_summary(idx):
    dialogue = datasets["test"][idx]["dialogue"]
    ref_summary = datasets["test"][idx]["summary"]
    summary = summarizer("summarize: "+datasets["test"][idx]["dialogue"])[0]["generated_text"]
    baseline = summarizer_base("summarize: "+datasets["test"][idx]["dialogue"])[0]["generated_text"]
    print(f"'>>> Dialogue: {dialogue}'")
    print(f"\n'>>> Summary: {summary}'")
    print(f"\n'>>> Reference summary: {ref_summary}'")
    print(f"\n'>>> Baseline (no fine tuning): {baseline}'")

In [29]:
print_summary(100)

Token indices sequence length is longer than the specified maximum sequence length for this model (623 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (623 > 512). Running this sequence through the model will result in indexing errors


'>>> Dialogue: The British Transport Police said the move was a "proportionate response" in the face of a mounting terrorism threat.
Specially trained officers will begin carrying the stun weapons over the next few weeks.
It brings the Scottish force into line with their counterpart in England, where Tasers have been used since 2011.
The weapons are used to incapacitate suspects through the use of an electric current.
Temporary Assistant Chief Constable Alun Thomas said: "This decision is not based on specific intelligence of any criminal behaviour or imminent threat, but will allow us the option to deploy Taser devices where, in the course of their duty, an officer needs to protect the public or themselves by using force.
"The current threat to the UK from international terrorism remains 'severe', meaning an attack is highly likely.
"Recent terrorist attacks across the world are a stark reminder that the threat from terrorism is a genuine risk, and it is important that we keep our sec

## Rouge SEM scoring

In [44]:
!git clone https://github.com/zhangming-19/ROUGE-SEM.git

Cloning into 'ROUGE-SEM'...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 63 (delta 17), reused 47 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 18.54 KiB | 9.27 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [30]:
!git clone https://github.com/summanlp/evaluation

Cloning into 'evaluation'...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


remote: Enumerating objects: 2075, done.
remote: Total 2075 (delta 0), reused 0 (delta 0), pack-reused 2075 (from 1)
Receiving objects: 100% (2075/2075), 6.65 MiB | 22.46 MiB/s, done.
Resolving deltas: 100% (360/360), done.


In [37]:
!export ROUGE_EVAL_HOME="/kaggle/working/evaluation/ROUGE-RELEASE-1.5.5/data/"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
!pyrouge_set_rouge_path /kaggle/working/evaluation/ROUGE-RELEASE-1.5.5

2025-10-25 18:53:09,357 [MainThread  ] [INFO ]  Set ROUGE home directory to /kaggle/working/evaluation/ROUGE-RELEASE-1.5.5.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [78]:
def get_summaries(indexes):
    for idx in indexes:
        dialogue = datasets["test"][idx]["dialogue"]
        ref_summary = datasets["test"][idx]["summary"]
        summary = summarizer("summarize: "+datasets["test"][idx]["dialogue"])[0]["generated_text"]
        # baseline = summarizer_base("summarize: "+datasets["test"][idx]["dialogue"])[0]["generated_text"]

        with open("./candidate.txt", "a") as text_file:
            text_file.write(summary)

        with open("./reference.txt", "a") as text_file:
            text_file.write(ref_summary)

In [88]:
!rm candidate.txt
!rm reference.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [89]:
get_summaries([idx for idx in range(0,5)])

In [91]:
!cat reference.txt

There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.
A man has appeared in court after firearms, ammunition and cash were seized by police in Edinburgh.
Four people accused of kidnapping and torturing a mentally disabled man in a "racially motivated" attack streamed on Facebook have been denied bail.
West Brom have appointed Nicky Hammond as technical director, ending his 20-year association with Reading.
The pancreas can be triggered to regenerate itself through a type of fasting diet, say US researchers.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [90]:
!cat candidate.txt

A charity has said it is "a desperate need" to find suitable accommodation for those leaving prison, a charity has said, in a bid to save the public purse of more than a hundred pounds a week, in the wake of changes to the Housing Act in Wales.
A man has appeared in court charged with the murder of a man in Edinburgh in connection with a shooting at a house in the city of Edinburgh, a police investigation has concluded...''Another man has appeared in court charged with the murder of a man in Edinburgh, a police investigation has concluded.
Four people have been charged with aggravated kidnapping and battery after a man allegedly beat a white man in a van in Chicago, a court has heard, according to a police investigation into the incident.
West Brom have signed West Brom striker David O'Neill on a two-year deal... CLICK HERE for all the latest football news from West Brom news, including football news, football news, football news and football news from West Brom.
People with type 1 and

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [96]:
!python3 /kaggle/working/ROUGE-SEM/calculate_lexical_similarity.py -r reference.txt -c candidate.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
################### Total #################
Rouge
ROUGE 1-2-L F: 28.00-11.97-28.58-
################### Each #################
ROUGE 1-2-L F: 36.92- 9.52-31.62-
ROUGE 1-2-L F: 25.71-17.65-32.89-
ROUGE 1-2-L F: 34.48-17.86-34.29-
ROUGE 1-2-L F: 14.29- 7.41-20.02-
ROUGE 1-2-L F: 28.57- 7.41-24.06-


In [97]:
lex_sim = pd.read_csv("/kaggle/working/lexical_similarity.csv")
lex_sim

,Unnamed: 0,ref,can,R1,R2,RL,lex_score
0,0,"there is a ""chronic"" need for more housing for...","a charity has said it is ""a desperate need"" to...",36.92,9.52,31.62,26.580
1,1,"a man has appeared in court after firearms, am...",a man has appeared in court charged with the m...,25.71,17.65,32.89,26.164
2,2,four people accused of kidnapping and torturin...,four people have been charged with aggravated ...,34.48,17.86,34.29,29.418
3,3,west brom have appointed nicky hammond as tech...,west brom have signed west brom striker david ...,14.29,7.41,20.02,14.518
4,4,the pancreas can be triggered to regenerate it...,people with type 1 and type 2 diabetes should ...,28.57,7.41,24.06,20.418


In [105]:
!cp -r /kaggle/working/flan-t5-xsum/checkpoint-6250/* /kaggle/working/ROUGE-SEM/model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [119]:
!python3 /kaggle/working/ROUGE-SEM/calculate_semantic_similarity.py -r reference.txt -c candidate.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2025-10-25 20:27:48.438026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761424068.460485     591 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761424068.467241     591 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|██████████████████████████████████████████| 5/5 [00:00<00:00, 10225.02it/s]


In [120]:
sem_sim = pd.read_csv("/kaggle/working/semantic_similarity.csv")
sem_sim

,ref,can,sem_score
0,"there is a ""chronic"" need for more housing for...","a charity has said it is ""a desperate need"" to...",0.877895
1,"a man has appeared in court after firearms, am...",a man has appeared in court charged with the m...,0.632784
2,four people accused of kidnapping and torturin...,four people have been charged with aggravated ...,0.610834
3,west brom have appointed nicky hammond as tech...,west brom have signed west brom striker david ...,0.743915
4,the pancreas can be triggered to regenerate it...,people with type 1 and type 2 diabetes should ...,0.789195


In [133]:
!python3 /kaggle/working/ROUGE-SEM/candidate_summary_classifier.py -lex_score lexical_similarity.csv -sem_score semantic_similarity.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [134]:
cat_sum = pd.read_csv("/kaggle/working/categorized_summary.csv")
cat_sum

,Unnamed: 0,ref,can,lex_score,sem_score,category
0,0,"there is a ""chronic"" need for more housing for...","a charity has said it is ""a desperate need"" to...",26.580,0.877895,2
1,1,"a man has appeared in court after firearms, am...",a man has appeared in court charged with the m...,26.164,0.632784,2
2,2,four people accused of kidnapping and torturin...,four people have been charged with aggravated ...,29.418,0.610834,2
3,3,west brom have appointed nicky hammond as tech...,west brom have signed west brom striker david ...,14.518,0.743915,2
4,4,the pancreas can be triggered to regenerate it...,people with type 1 and type 2 diabetes should ...,20.418,0.789195,2


In [132]:
%%writefile /kaggle/working/ROUGE-SEM/candidate_summary_classifier.py
# -*- coding: utf-8 -*-
"""
Created on Thu Mar  2 20:53:19 2023

@author: zhangming
"""
import os
import torch
import argparse
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import tqdm
import csv
import numpy as np

Alpha_parameter = 0.5
Beta_parameter = 0.5

parser = argparse.ArgumentParser()
parser.add_argument('-lex_score', type=str, default="lexical_similarity.csv",
                    help='candidate file')
parser.add_argument('-sem_score', type=str, default="semantic_similarity.csv",
                    help='reference file')
args = parser.parse_args()

lex_df = pd.read_csv(args.lex_score)
lex_score_list = lex_df["lex_score"].tolist()
references = lex_df["ref"].tolist()
candidates = lex_df["can"].tolist()

sem_df = pd.read_csv(args.sem_score)
sem_score_list = sem_df["sem_score"].tolist()    

category_list = []
for item_lex, item_sem in zip(lex_score_list, sem_score_list):
    item_lex = item_lex / 100.0 # normalize lexical score
    if item_lex >= Alpha_parameter and item_sem >= Beta_parameter:
        category_list.append(0)
    if item_lex >= Alpha_parameter and item_sem < Beta_parameter:
        category_list.append(1)
    if item_lex < Alpha_parameter and item_sem >= Beta_parameter:
        category_list.append(2)
    if item_lex < Alpha_parameter and item_sem < Beta_parameter:
        category_list.append(3)

name = ['ref', 'can', 'lex_score', 'sem_score', 'category']
temp = []
temp.append(references)
temp.append(candidates)
temp.append(lex_score_list)
temp.append(sem_score_list)
temp.append(category_list)
temp_df = np.array(temp)
temp_df = temp_df.T
temp_df = pd.DataFrame(temp_df, columns=name)
temp_df.to_csv('categorized_summary.csv', encoding='utf-8')
            

Overwriting /kaggle/working/ROUGE-SEM/candidate_summary_classifier.py


In [118]:
%%writefile /kaggle/working/ROUGE-SEM/calculate_semantic_similarity.py
# -*- coding: utf-8 -*-
"""
Created on Thu Mar  2 20:53:19 2023

@author: zhangming
"""
import os
import torch
import argparse
from scipy.spatial.distance import cosine
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
import tqdm
import csv

def get_sents_str(file_path):
    sents = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            line = line.strip()
            line = line.lower()
            sents.append(line)
    return sents

parser = argparse.ArgumentParser()
parser.add_argument('-c', type=str, default="candidate.txt",
                    help='candidate file')
parser.add_argument('-r', type=str, default="reference.txt",
                    help='reference file')
args = parser.parse_args()

ref_path = get_sents_str(args.r)
can_path = get_sents_str(args.c)
    
# Usa il percorso assoluto per evitare problemi con Hugging Face
model_path = "/kaggle/working/ROUGE-SEM/model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Usa DIRETTAMENTE le liste (BUG CORRETTO)
ref_summary = ref_path  # Questa è già una lista di stringhe
can_summary = can_path  # Questa è già una lista di stringhe
    
# Tokenize input texts
ref_inputs = tokenizer(ref_summary, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings DALL'ENCODER (non dal modello completo)
with torch.no_grad():
    ref_outputs = model.get_encoder()(**ref_inputs)
    # Usa last_hidden_state e fai mean pooling
    ref_embeddings = ref_outputs.last_hidden_state.mean(dim=1)
    
# Tokenize candidate texts
can_inputs = tokenizer(can_summary, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    can_outputs = model.get_encoder()(**can_inputs)
    # Usa last_hidden_state e fai mean pooling
    can_embeddings = can_outputs.last_hidden_state.mean(dim=1)

# Calculate cosine similarities
similar_lst = []
for i in tqdm.tqdm(range(len(can_embeddings))):
    cosine_sim_ref_can = 1 - cosine(ref_embeddings[i].numpy(), can_embeddings[i].numpy())
    similar_lst.append(cosine_sim_ref_can)
                           
data_list = []
for a, b, c in zip(ref_summary, can_summary, similar_lst):
    x = {}
    x['ref'] = a
    x['can'] = b
    x['Sem'] = c
    data_list.append(x)

outpath = './semantic_similarity.csv'
with open(outpath, 'w', newline='', encoding='UTF-8') as f_c_csv:
    writer = csv.writer(f_c_csv)
    writer.writerow(['ref', 'can', 'sem_score'])
    for nl in data_list:
        writer.writerow(nl.values())

Overwriting /kaggle/working/ROUGE-SEM/calculate_semantic_similarity.py


In [95]:
%%writefile /kaggle/working/ROUGE-SEM/calculate_lexical_similarity.py
import sys
import os
import rouge
import bert_score
import argparse
import codecs
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd


class Logger(object):
    def __init__(self, filename="Default.log"):
        self.terminal = sys.stdout
        self.log = open(filename, "a")
 
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
 
    def flush(self):
        pass


def prepare_results(metric, p, r, f):
    return '\t{}:\t {:5.2f}\t {:5.2f}\t {:5.2f}'.format(metric, 100.0 * p, 100.0 * r,
                                                                 100.0 * f)


def test_rouge(candidates, references):
    candidates = [line.strip() for line in candidates]
    references = [line.strip() for line in references]
    assert len(candidates) == len(references)

    apply_avg = True
    apply_best = False

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                            max_n=4,
                            limit_length=True,
                            length_limit=100,
                            length_limit_type='words',
                            apply_avg=apply_avg,
                            apply_best=apply_best,
                            alpha=0.5,  # Default F1_score
                            weight_factor=1.2,
                            stemming=True)

    all_hypothesis = candidates
    all_references = references

    scores = evaluator.get_scores(all_hypothesis, all_references)
#    print(scores)

    rougel = ""
    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if metric in ["rouge-1", "rouge-2", "rouge-l"]:
    #        print(prepare_results(metric, results['p'], results['r'], results['f']))
            rougel = rougel + '{:5.2f}'.format(100 * results['f']) + "-"

    print("ROUGE 1-2-L F:", rougel)
    return rougel



def get_sents_str(file_path):
    sents = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            line = line.strip()
            line = line.lower()
            sents.append(line)
    return sents


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('-c', type=str, default="candidate.txt",
                        help='candidate file')
    parser.add_argument('-r', type=str, default="reference.txt",
                        help='reference file')
    args = parser.parse_args()

    references = get_sents_str(args.r)
    candidates = get_sents_str(args.c)

    #print(references)
    sys.stdout = Logger('test_log.txt')
    
    print('################### Total #################')
    print('Rouge')
    score_total = test_rouge(candidates, references)
    
    print('################### Each #################')
    R1_list = []
    R2_list = []
    RL_list = []

    for item_c, item_r in zip(candidates, references):
        # 01 Rouge score - CORRETTO
        score_tmp = test_rouge([item_c], [item_r])
        score_parts = score_tmp.rstrip('-').split('-')  # Rimuovi trattino finale
        R1_list.append(float(score_parts[0]))
        R2_list.append(float(score_parts[1]))
        RL_list.append(float(score_parts[2]))
    
    lex_score_list = []
    for item_R1, item_R2, item_RL in zip(R1_list, R2_list, RL_list):
        lex_score_temp = 0.3*float(item_R1) + 0.3*float(item_R2) + 0.4*float(item_RL)
        lex_score_list.append(lex_score_temp)

    # Rouge + Bert 
    name = ['ref', 'can', 'R1', 'R2', 'RL', 'lex_score']
    temp = []
    temp.append(references)
    temp.append(candidates)
    temp.append(R1_list)
    temp.append(R2_list)
    temp.append(RL_list)
    temp.append(lex_score_list)
    temp_df = np.array(temp)
    temp_df = temp_df.T
    temp_df = pd.DataFrame(temp_df, columns=name)
    temp_df.to_csv('lexical_similarity.csv', encoding='utf-8')

Overwriting /kaggle/working/ROUGE-SEM/calculate_lexical_similarity.py
